<a href="https://colab.research.google.com/github/bluecityisu/class2022/blob/main/Lstm_RF_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [248]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [249]:
def create_dataset(dataset, look_back):
 dataX, dataY = [], []
 for i in range(len(dataset)-look_back):
  a = dataset.iloc[i:(i+look_back), 1:]
  dataX.append(a)
  dataY.append(dataset.iloc[i + look_back, 0])
 return np.array(dataX), np.array(dataY)

In [250]:
url='/content/drive/MyDrive/專題2024/ran_forest/20220121_20230609.xlsx 的副本'
data = pd.read_excel(url)

data.tail()

,t,Y9999,Y99992,2302 麗正,2303 聯電,2329 華泰,2330 台積電,2337 旺宏,2338 光罩,2340 台亞,...,8081 致新,8086 宏捷科,8088 品安,8110 華東,8131 福懋科,8150 南茂,8261 富鼎,8271 宇瞻,8277 商丞,8299 群聯
66,2023-05-12,-0.007917,0,-0.017588,-0.018145,-0.040816,-0.008000,-0.018462,-0.045906,-0.033803,...,0.088068,-0.009988,-0.030675,-0.045113,-0.012972,-0.009186,-0.028986,-0.041833,-0.026396,-0.010458
67,2023-05-19,0.043384,1,-0.023018,0.026694,0.021277,0.072581,0.028213,0.036411,0.037901,...,-0.013055,-0.016393,0.050633,0.031496,0.015532,0.017219,0.044776,0.072765,0.008342,0.026420
68,2023-05-26,0.020410,1,0.013089,0.008000,0.031250,0.063910,0.004573,0.010038,0.000000,...,0.005291,0.042308,-0.030120,0.007634,0.004706,0.028646,-0.004762,0.011628,-0.001034,0.024453
69,2023-06-02,0.012230,1,0.082687,0.019841,0.065657,-0.007067,0.027314,-0.002484,0.030899,...,0.013158,0.083641,0.133540,0.045455,0.026932,0.016456,0.033493,0.201149,0.221532,0.031407
70,2023-06-09,0.010743,1,0.026253,0.017510,0.090047,0.005338,-0.051699,0.032379,0.021798,...,0.028571,0.024972,-0.027397,0.000000,-0.003421,-0.002491,-0.004630,-0.030303,-0.004237,0.025579


## **chapter 2 隨機森林**

In [251]:
data2=data.drop(['t'], axis=1)
data2=data2.drop(['Y9999'], axis=1)
data2.iloc[:,0]

0     0
1     0
2     1
3     0
4     0
     ..
66    0
67    1
68    1
69    1
70    1
Name: Y99992, Length: 71, dtype: int64

In [252]:
look_back = 1
x, y = create_dataset(data2, look_back)
x=np.reshape(x, (x.shape[0],x.shape[2]))
x.shape

(70, 153)

In [253]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clfr = RandomForestClassifier(max_depth=3, n_jobs=-1)
clfr.fit(x, y)   #開始計算
y_pred = clfr.predict(x)
#print(classification_report(y, y_pred))

In [254]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
# 建立特徵選取器，門檻值預設為重要性的平均值
selectorr = SelectFromModel(clfr)
selectorr.fit(x, y)
print('門檻值 =', selectorr.threshold_)
print('特徵遮罩：', selectorr.get_support())

# 選出新特徵，重新訓練隨機森林
x_new = selectorr.transform(x)
x_new=pd.DataFrame(x_new)
y=pd.DataFrame(y)

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42).split(x, y)
score_lst = []  # 紀錄 k 次交叉驗證的正確率

for k, (i_train, i_valid) in enumerate(kfold):
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])

  y_pred_test = clfr.predict(x_new.iloc[i_valid, :])

  score=accuracy_score(y.iloc[i_valid], y_pred_test)
  print('precision=',score)
  score_lst.append(score)
  print('%2d-Fold: Acc=%.3f' % (k+1, score))

print('\n10-fold CV accuracy = %.3f, std = %.3f' % (np.mean(score_lst), np.std(score_lst)))
print(np.mean(score_lst)/np.std(score_lst))

門檻值 = 0.006535947712418301
特徵遮罩： [False False False  True False  True False False  True False False False
  True False False  True  True  True False False False False  True  True
 False False  True False  True  True  True  True False  True  True False
 False False False  True False False False False False False False False
 False  True False False False  True  True  True  True False False  True
  True False False False False False  True  True False False  True False
  True False False False  True  True  True False  True  True False  True
  True False False False False False False False False  True False  True
 False False False  True False  True  True False False False  True  True
  True False False False False False  True  True False  True False False
  True False False False False False False False False False  True  True
  True False False False False False False False False  True False  True
 False False False False False False False False False]


<ipython-input-254-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.42857142857142855
 1-Fold: Acc=0.429


<ipython-input-254-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.5714285714285714
 2-Fold: Acc=0.571


...

precision= 0.7142857142857143
 8-Fold: Acc=0.714


<ipython-input-254-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.5714285714285714
 9-Fold: Acc=0.571


<ipython-input-254-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.7142857142857143
10-Fold: Acc=0.714

10-fold CV accuracy = 0.600, std = 0.107
5.612486080160911


# **chapter 3 LSTM**

In [255]:
data2=data.drop(['t'], axis=1)
data2=data2.drop(['Y9999'], axis=1)
y=data2.pop("Y99992")
x = selectorr.transform(data2)
x=pd.DataFrame(x)
data2=pd.concat([y,x], axis=1)
data2

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


,Y99992,0,1,2,3,4,5,6,7,8,...,42,43,44,45,46,47,48,49,50,51
0,0,-0.046131,0.023590,0.010101,0.022556,-0.026918,-0.018519,-0.005924,0.025316,-0.006719,...,-0.023753,-0.037037,-0.111628,-0.043228,-0.015796,-0.014060,-0.065574,0.010204,0.014610,0.017094
1,0,-0.007800,-0.033066,-0.001667,-0.033088,-0.004149,-0.016173,0.036949,-0.003086,0.020293,...,-0.051095,-0.019231,0.009162,-0.054217,-0.024691,-0.023173,-0.003509,-0.043771,-0.036800,-0.033613
2,1,0.022013,0.072539,0.105175,-0.041825,0.097222,0.046575,0.070115,0.058824,0.160221,...,0.110256,0.090196,-0.046693,-0.001592,-0.010127,0.010949,0.137324,0.056338,0.073090,0.041304
3,0,-0.020000,-0.004831,0.086103,-0.041667,0.056962,-0.032723,0.024705,0.020468,0.038095,...,-0.092379,0.089928,-0.027211,0.000000,0.002558,-0.041516,-0.009288,-0.016667,-0.021672,0.079332
4,0,-0.051805,-0.048544,-0.082058,-0.055901,-0.076647,0.013532,-0.071279,-0.060172,-0.077982,...,-0.040712,-0.092409,-0.037762,0.001595,-0.076531,-0.054614,-0.081250,-0.016949,-0.042722,-0.079304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0,-0.008000,-0.045906,-0.030426,0.008197,-0.043478,0.015000,-0.037037,0.001044,-0.012012,...,-0.037037,-0.025000,-0.031315,-0.002112,0.010601,0.000000,-0.030635,0.094527,-0.066813,0.088068
67,1,0.072581,0.036411,0.087866,0.018970,0.095611,-0.021346,0.043803,0.027112,0.018237,...,0.019231,0.015385,0.044181,-0.001058,0.089161,0.023438,-0.001129,-0.072727,0.058685,-0.013055
68,1,0.063910,0.010038,0.001923,0.007979,0.028612,0.030201,0.017400,0.040609,0.022388,...,0.004717,0.003030,0.029928,0.042373,-0.003210,0.045802,0.023729,-0.004902,-0.013304,0.005291
69,1,-0.007067,-0.002484,0.094050,0.021108,0.013908,0.030945,0.070423,0.019512,0.036496,...,0.023474,0.092649,0.012024,0.011179,0.082126,0.036496,0.011038,-0.004926,0.114607,0.013158


In [256]:
train_size = int(len(data2) * 0.67)
test_size = len(data2) - train_size
train, test = data2.iloc[0:train_size,:], data2.iloc[train_size:len(data2),:]
train.tail()

,Y99992,0,1,2,3,4,5,6,7,8,...,42,43,44,45,46,47,48,49,50,51
42,1,0.103058,0.071429,0.002288,0.068220,0.015050,0.041739,0.017978,0.017222,0.032702,...,0.035870,0.049226,0.034783,0.029621,0.156695,0.037681,-0.011538,-0.003822,-0.048333,-0.024055
43,1,0.022587,0.108333,-0.011416,0.029595,-0.016474,0.001669,0.007726,0.000000,0.006667,...,-0.018888,-0.048257,0.008403,0.070196,0.022167,0.094972,-0.044747,-0.015345,-0.026270,0.007042
44,1,-0.011044,0.199248,0.016166,-0.013616,-0.060302,-0.008333,0.065717,0.038375,0.087748,...,0.085561,0.108451,0.018750,0.169892,0.007229,0.089286,0.120163,0.083117,0.064748,0.066434
45,0,-0.022335,-0.031348,-0.059091,-0.081288,-0.042781,-0.038655,-0.075026,-0.031522,-0.053272,...,-0.072906,-0.083863,-0.053170,-0.040441,-0.107656,-0.091335,-0.087273,-0.058753,-0.070946,-0.065574
46,0,-0.021807,-0.029126,0.007246,-0.021703,0.014898,-0.010490,-0.018889,0.013468,0.028939,...,-0.027630,-0.012483,-0.016199,-0.003831,-0.029491,-0.033505,-0.013944,-0.019108,-0.032727,0.035088


In [257]:
look_back = 4
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainY

array([0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0])

In [258]:
from keras.utils import to_categorical
trainY = to_categorical(trainY, dtype ="uint8")
testY = to_categorical(testY, dtype ="uint8")


In [259]:
trainY.shape

(43, 2)

In [260]:
model = Sequential([
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(4,input_shape=[look_back, trainX.shape[2]],return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
  Dense(64, activation='relu'),
  Dense(32, activation='relu'),
  Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [261]:
model.fit(trainX, trainY, epochs=100, verbose=2)

Epoch 1/100
2/2 - 10s - loss: 0.6939 - accuracy: 0.4651 - 10s/epoch - 5s/step
Epoch 2/100
2/2 - 0s - loss: 0.6927 - accuracy: 0.6047 - 30ms/epoch - 15ms/step
Epoch 3/100
2/2 - 0s - loss: 0.6919 - accuracy: 0.6744 - 21ms/epoch - 11ms/step
Epoch 4/100
2/2 - 0s - loss: 0.6914 - accuracy: 0.5581 - 21ms/epoch - 10ms/step
Epoch 5/100
2/2 - 0s - loss: 0.6907 - accuracy: 0.5349 - 21ms/epoch - 10ms/step
Epoch 6/100
2/2 - 0s - loss: 0.6900 - accuracy: 0.5581 - 23ms/epoch - 11ms/step
Epoch 7/100
2/2 - 0s - loss: 0.6898 - accuracy: 0.5349 - 20ms/epoch - 10ms/step
Epoch 8/100
2/2 - 0s - loss: 0.6888 - accuracy: 0.5349 - 20ms/epoch - 10ms/step
Epoch 9/100
2/2 - 0s - loss: 0.6883 - accuracy: 0.5814 - 20ms/epoch - 10ms/step
Epoch 10/100
2/2 - 0s - loss: 0.6875 - accuracy: 0.5814 - 20ms/epoch - 10ms/step
Epoch 11/100
2/2 - 0s - loss: 0.6867 - accuracy: 0.5814 - 19ms/epoch - 10ms/step
Epoch 12/100
2/2 - 0s - loss: 0.6860 - accuracy: 0.5814 - 21ms/epoch - 10ms/step
Epoch 13/100
2/2 - 0s - loss: 0.6851 - 

In [262]:
testPredict = model.predict(testX)
#testPredict=np.reshape(testPredict, -1)
print(testY)
print(testPredict)

1/1 [==============================] - 1s 1s/step
[[0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
[[8.1666052e-01 1.8333951e-01]
 [8.7683487e-01 1.2316516e-01]
 [9.7075528e-01 2.9244700e-02]
 [9.8378223e-01 1.6217716e-02]
 [9.8574936e-01 1.4250660e-02]
 [7.5748754e-01 2.4251249e-01]
 [6.5489042e-01 3.4510958e-01]
 [9.9909484e-01 9.0511574e-04]
 [9.9892104e-01 1.0790491e-03]
 [9.9864286e-01 1.3571138e-03]
 [9.9890924e-01 1.0907911e-03]
 [9.9791878e-01 2.0811893e-03]
 [9.9953640e-01 4.6359657e-04]
 [9.9937505e-01 6.2496657e-04]
 [9.9983549e-01 1.6442797e-04]
 [9.9966991e-01 3.3010711e-04]
 [9.8688626e-01 1.3113740e-02]
 [8.2690942e-01 1.7309059e-01]
 [4.6505362e-01 5.3494638e-01]
 [9.9053383e-01 9.4661592e-03]]


In [263]:
def deter(x):
  de=[]
  for i in range(len(x)):
    if x[i][0]>x[i][1]:
      de.append(0)
    else:
      de.append(1)
  return de

In [264]:
testY1=deter(testY)
testPredict1=deter(testPredict)
print(testY1)
print(testPredict1)

[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [265]:
from sklearn.metrics import accuracy_score
score=accuracy_score(testY1, testPredict1)
score

0.5

In [266]:
test.iloc[:,1:]

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
47,-0.033970,-0.041111,-0.035971,-0.013652,-0.056881,0.000000,-0.048698,-0.046512,-0.056250,-0.044295,...,-0.063388,-0.073034,-0.043908,-0.056731,-0.060773,0.010667,-0.082828,-0.006494,0.028195,-0.044068
48,-0.014286,-0.018540,-0.024876,-0.027682,-0.003891,-0.017668,-0.016667,-0.001161,-0.041391,-0.014045,...,-0.017503,-0.021212,-0.018370,-0.050968,-0.029412,-0.052770,-0.017621,0.026144,-0.040219,0.010638
49,0.022297,0.038961,0.030612,0.092527,0.074219,0.003597,0.054479,0.097674,0.044905,0.038462,...,-0.003563,0.058824,0.040936,0.035446,0.103030,0.055710,0.066143,-0.003822,0.005714,0.066667
50,0.090513,0.031818,0.024752,0.034202,0.018182,0.003584,0.064294,-0.039195,0.021488,-0.021948,...,0.028605,0.026316,0.092135,0.020747,0.005495,-0.013193,-0.062040,-0.002558,0.001894,-0.006579
51,0.006000,-0.028634,0.014493,-0.003150,0.030357,-0.007143,-0.018339,0.003308,0.016181,0.009818,...,0.002317,0.021368,0.014403,0.002033,0.024590,0.008021,0.020179,0.005128,-0.028355,0.000000
52,0.077535,0.020408,0.107143,0.099526,0.038128,0.010791,0.118681,0.181319,0.100318,0.083333,...,0.231214,0.046025,0.099391,0.123732,0.320000,0.092838,0.151648,0.172194,0.058366,0.092715
53,0.005535,-0.047778,-0.038710,0.025862,-0.026711,0.008897,-0.071709,-0.013953,-0.030391,-0.019231,...,0.098592,-0.009333,-0.020295,0.054152,-0.002020,0.021845,-0.041985,-0.011970,-0.029412,0.009091
54,-0.049541,0.012835,0.035794,0.007003,0.024014,-0.022928,0.048677,-0.028302,-0.001493,-0.006536,...,0.098291,0.059219,-0.026365,-0.010274,0.052632,0.009501,0.023904,-0.013216,-0.005682,0.021021
55,-0.013514,-0.048387,-0.004320,0.057024,-0.021776,0.000000,0.021191,0.053398,-0.001495,-0.005263,...,0.003891,-0.043202,-0.009671,0.019031,0.088462,-0.014118,0.023346,0.033482,-0.011429,0.029412
56,0.009785,0.003632,0.110629,-0.001316,-0.008562,0.009025,0.023715,-0.041475,-0.004491,-0.048942,...,-0.015504,0.059761,-0.015625,0.010187,0.000000,-0.011933,0.001901,-0.001080,0.013487,0.011429


In [267]:
recent=test.iloc[test.shape[0]-look_back:test.shape[0], 1:]
recent=np.reshape(recent, (1, look_back, test.shape[1]-1))
recent

array([[[ 0.07258065,  0.03641092,  0.08786611,  0.01897019,
          0.09561129, -0.02134647,  0.04380342,  0.02711157,
          0.01823708,  0.0192926 ,  0.02994012, -0.0027027 ,
          0.0828125 , -0.04716981,  0.00512295,  0.00858369,
          0.10114192,  0.09401709,  0.03088803,  0.03092784,
          0.07951482, -0.00749064, -0.03198294,  0.1391106 ,
          0.03311258,  0.05825243,  0.02218115, -0.01744186,
          0.00519481,  0.02834467,  0.05128205,  0.0718232 ,
         -0.01221167,  0.02236422,  0.02232143,  0.00465116,
         -0.00666667,  0.0148368 ,  0.00172414,  0.06102362,
          0.01890359,  0.01408451,  0.01923077,  0.01538462,
          0.04418103, -0.0010582 ,  0.08916084,  0.0234375 ,
         -0.00112867, -0.07272727,  0.05868545, -0.01305483],
        [ 0.06390977,  0.01003764,  0.00192308,  0.00797872,
          0.0286123 ,  0.03020134,  0.0174002 ,  0.04060914,
          0.02238806, -0.00630915, -0.02034884,  0.04878049,
         -0.02308802,  

# **chapter 4 決鬥**

In [268]:
x = data.iloc[-1:, 3:]  #最後一筆x
x_new = selectorr.transform(x)   #隨機森林選出的x，隨機森林比較大就選這個
y_pred_final = clfr.predict(x_new)
y_pred_final

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


array([1])

In [269]:
recentP = model.predict(recent)  #LSTM比較大就選這個
print(deter(recentP))


1/1 [==============================] - 0s 20ms/step
[0]
